# Подготовка данных для Проектного практикума - Учебная задача (семестр 3)

## Установка зависимостей (если ранее не установлены)

In [1]:
# Установка основных зависимостей
# %pip install pandas num2words wget

In [2]:
# Установка зависимостей для блокнота
# %pip install ipywidgets jupyter IProgress tqdm 

## Импорты, константы и настройки

In [1]:
# Импорт библиотек 
import os
import pathlib
import re
import wget

import pandas as pd

from num2words import num2words # Числа в текст

from tqdm import tqdm   # Прогресс бар

In [4]:
# Константы
DATASET_URL: str = 'https://github.com/yandex/geo-reviews-dataset-2023/raw/refs/heads/master/geo-reviews-dataset-2023.tskv?download='

RAW_DATASET_PATH: str = '.././data/raw'            # '..' из-за того, что блокнот не руте
PARSED_DATASET_PATH: str = '.././data/parsed'      # '..' из-за того, что блокнот не руте
CLEARED_DATASET_PATH: str = '.././data/cleared'    # '..' из-за того, что блокнот не руте

In [5]:
# Создание структуры каталогов для данных
def create_paths(fullpath: str) -> bool:
    '''Создание каталогов с подкаталогами'''
    os.makedirs(fullpath, exist_ok=True)
    return os.path.exists(fullpath)

print(f'Путь {RAW_DATASET_PATH} создан/существует: {create_paths(RAW_DATASET_PATH)}')
print(f'Путь {PARSED_DATASET_PATH} создан/существует: {create_paths(PARSED_DATASET_PATH)}')
print(f'Путь {CLEARED_DATASET_PATH} создан/существует: {create_paths(CLEARED_DATASET_PATH)}')

Путь ./data/raw создан/существует: True
Путь ./data/parsed создан/существует: True
Путь ./data/cleared создан/существует: True


In [66]:
# Импорт програссбара для Pandas
tqdm.pandas()

## Загрузка данных

Можно закомментировать после первого запуска

In [8]:
# Загрузка данных
def bar_progress(current, total, width=80):
    '''Функция прогрессбара загрузки'''
    import sys
    progress_message = "Загрузка: %d%% [%d / %d] байт" % (current / total * 100, current, total)
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

raw_dataset_path: str = wget.download(
    url=DATASET_URL,
    out=RAW_DATASET_PATH,
    bar=bar_progress)

raw_dataset_path: pathlib.Path = pathlib.Path(raw_dataset_path)

print(f'\n\n Файл датасета сохранен в: {raw_dataset_path}')

Загрузка: 100% [378730064 / 378730064] байт

 Файл датасета сохранен в: data\raw\geo-reviews-dataset-2023.tskv


In [10]:
# Путь к файлу и имя файла формируется выше если данные скачаны,
# но если не скачаны или закомментировано, то надо найти вручную
if 'raw_dataset_path' not in globals():
    raw_dataset_path = None # Такой хак для линтера
    globals()['raw_dataset_path'] = None

if not raw_dataset_path:
    files = pathlib.Path(RAW_DATASET_PATH).glob('*.tskv')
    if not files:
        raise RuntimeError('Данных с расширением tskv не найдено')
    raw_dataset_path: pathlib.Path = list(files)[0]

print(f'Путь к сырому датасету: {raw_dataset_path}')

Путь к сырому датасету: data\raw\geo-reviews-dataset-2023.tskv


## Чтение данных

In [36]:
## Функция парсинга формата tskv
def read_tskv(filepath: pathlib.Path) -> list[dict[str, str]]:
    '''Парсинг файлов формата tskv'''

    if not str(filepath).lower().endswith('tskv'):
        raise RuntimeError(f'Формат файла {str(filepath)} не подходит')
    
    data: list[dict[str, str]] = list()
    try:
        with open(filepath, 'r', encoding='utf-8') as tskv_file:
            for line in tskv_file:
                # Удаление пробелов и проспусков строк
                line = line.strip()
                if not line:
                    continue

                # Пара ключ-значение: разбиваем по табуляции 
                # и затем по знаку равенства
                record: dict[str, str] = {}
                fields = line.split('\t')
                for field in fields:
                    if '=' in field:
                        key, value = field.split('=', 1)
                        record[key] = value
                data.append(record)
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
    return data

In [37]:
# Чтение файла и получение DataFrame
df_data: pd.DataFrame = pd.DataFrame(read_tskv(raw_dataset_path))
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  500000 non-null  object
 1   name_ru  499030 non-null  object
 2   rating   500000 non-null  object
 3   rubrics  500000 non-null  object
 4   text     500000 non-null  object
dtypes: object(5)
memory usage: 19.1+ MB


In [38]:
# Что внутри
df_data.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [39]:
# Сохранение распарсенного датасета
df_data.to_csv(
    os.path.join(PARSED_DATASET_PATH, 'parsed_data.csv'),
    sep=';',
    encoding='utf-8-sig',
    index_label='indx')

## Очистка данных

In [40]:
# Адрес и имя заведения нам не важно (скорее всего!)
df_data_clear = df_data.drop(['address', 'name_ru'], axis=1)
df_data_clear.head(n=5)

,rating,rubrics,text
0,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [41]:
# Получение количества пропусков
df_data_clear.isna().sum()

rating     0
rubrics    0
text       0
dtype: int64

### Обработка рейтинга (оценок)

In [42]:
# Константы
CLEAR_RATING_PATTERN: str = r'[^0-9]+'

In [43]:
# Функция удаления лишьних символов в рейтинге
def clear_rating(rating) -> int:
    if type(rating) is int:
        return rating
    if not rating: 
        return 0
    # Удаление лишьних символов
    val_str = re.sub(CLEAR_RATING_PATTERN, '', rating)
    if not val_str:
        return 0
    return int(val_str)

In [44]:
# Очистка и подготовка рейтинга
df_data_clear['rating_int'] = df_data_clear['rating'].apply(clear_rating)
df_data_clear = df_data_clear[df_data_clear['rating_int'] > 0]
df_data_clear.head()

,rating,rubrics,text,rating_int
0,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...,3
1,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...",5
2,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...",1
3,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...,4
4,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,5


### Обработка рубрик

In [45]:
# Константы
RUBRICS_SPLIT_SYMBOLS: str = r'[;,]'
CLEAR_RUBRIC_SYMBOLS: str = r'[^а-яА-Яa-zA-Z0-9ёЁ ]'

In [46]:
# Функция разделения рубрик в список
def clear_rubrics(rubrics: str) -> list[str]:
    rubrics_list: list[str] = re.split(RUBRICS_SPLIT_SYMBOLS, rubrics)
    for i in range(len(rubrics_list)):
        rubrics_list[i] = rubrics_list[i] \
            .lower() \
            .replace("\\n", " ") \
            .replace("\\r", " ")
        rubrics_list[i] = re.sub(CLEAR_RUBRIC_SYMBOLS, " ", rubrics_list[i])
        rubrics_list[i] = re.sub(r"\s+", " ", rubrics_list[i]).strip()
    return rubrics_list

In [47]:
# Очистка и подготовка рубрик
df_data_clear['rubrics_list'] = df_data_clear['rubrics'].apply(clear_rubrics)
df_data_clear.head()

,rating,rubrics,text,rating_int,rubrics_list
0,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...,3,[жилой комплекс]
1,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...",5,"[магазин продуктов, продукты глубокой заморозк..."
2,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...",1,[фитнес клуб]
3,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...,4,"[пункт проката, прокат велосипедов, сапсёрфинг]"
4,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,5,"[салон красоты, визажисты, стилисты, салон бро..."


### Обработка текста

In [48]:
# Константы для обработки дат
CLEAR_DATA_PATTERN_1: str = r'[\b]*((0?[1-9]|[12][0-9]|3[01])[-/.](0?[1-9]|1[0-2])[-/.]((19|20)?\d{2}))(г|г\.|год|год\.)?[\b]*'     # 17.12.2022
CLEAR_DATA_PATTERN_2: str = r'[\b]*(((19|20)?\d{2})[-/.](0?[1-9]|1[0-2])[-/.](0?[1-9]|[12][0-9]|3[01]))(г|г\.|год|год\.)?[\b]*'     # 2022.12.17
CLEAR_DATA_PATTERN_3: str = r'[\b]*(((0?[1-9]|1[0-2])[-/.](0?[1-9]|[12][0-9]|3[01])[-/.]((19|20)?\d{2})))(г|г\.|год|год\.)?[\b]*'   # 12/17/2022
CLEAR_DATA_PATTERN: str = r'[\b]*((0?[1-9]|[12][0-9]|3[01])[-/.](0?[1-9]|1[0-2])[-/.]((19|20)?\d{2})|((19|20)?\d{2})[-/.](0?[1-9]|1[0-2])[-/.](0?[1-9]|[12][0-9]|3[01])|((0?[1-9]|1[0-2])[-/.](0?[1-9]|[12][0-9]|3[01])[-/.]((19|20)?\d{2})))(г|г\.|год|год\.)?[.\b]*'
CLEAR_YEAR_PATTERN: str = r'[в]?\s+(?:(\d{2})|(\d{4}))\s*(?:году|год|г.)'

# Константы для обработки паттернов
CLEAR_AMOUNT_PATTERN: str = r'(?:\d{1,3}(?:[ ,]\d{3})*|\d+)(?:[.,]\d{1,2})?[\s]*(?:[₽$€]|(?:рублей|рубля|руб|р|долларов|евро))+[.]?'
CLEAR_TEXT_PATTERN_1: str = r'[0-9]+\s*из\s*[0-9]+'
CLEAR_TEXT_PATTERN_2: str = r'[0-9]+'
CLEAR_TEXT_REPLACER: str = r'[^а-яА-Я0-9ёЁ., ]'

In [49]:
# Настройки обработок
IS_LOWER_TEXT: bool = False
IS_E_YO_REPLACE: bool = True

IS_DATE_CLEAR_ENABLE: bool = True
IS_DATE_ONLY_DELETE: bool = True
IS_YEAR_CLEAR_ENABLE: bool = True
IS_YEAR_ONLY_DELETE: bool = False

IS_AMOUNT_CLEAR_ENABLE: bool = True
AMOUNT_CLEAR_CURRENCY: str = 'руб.'

IS_PATTERN_N_FROM_M_ENABLE: bool = True
IS_ADD_SCORE_TO_N_FROM_M: bool = True

IS_PATTERN_N_TO_STR_ENABLE: bool = True

IS_FINAL_CLEAR_ENABLE: bool = True

#### Служебные функции

In [50]:
# Замена текста в определенной позиции
def replace_positions(text: str, pos_from: int, pos_to: int, val: str) -> str:
    '''Замена текста в определенной позиции'''
    return f'{text[:pos_from]}{val}{text[pos_to:]}'

In [51]:
# Функция получения числа прописью в родительском падеже
def number_to_words_genitive(number):
    '''Функция получения родительного падежа прописью'''
    if number == 0:
        return 'нуля'
    units = ["", "одного", "двух", "трех", "четырех", "пяти", "шести", "семи", "восьми", "девяти"]
    teens = ["десяти", "одиннадцати", "двенадцати", "тринадцати", "четырнадцати", "пятнадцати", "шестнадцати", "семнадцати", "восемнадцати", "девятнадцати"]
    tens = ["", "десяти", "двадцати", "тридцати", "сорока", "пятидесяти", "шестидесяти", "семидесяти", "восьмидесяти", "девяноста"]
    hundreds = ["", "ста", "двухсот", "трёхсот", "четырёхсот", "пятисот", "шестисот", "семисот", "восьмисот", "девятисот"]
    thousands = ["", "одной тысячи", "двух тысяч", "трёх тысяч", "четырёх тысяч", "пяти тысяч", "шести тысяч", "семи тысяч", "восьми тысяч", "девяти тысяч"]

    if not (0 < number < 1000000):
        raise ValueError(f"Число должно {number} быть в диапазоне от 1 до 999999 включительно.")

    def get_hundreds_part(n):
        if n == 0:
            return ""
        elif n < 10:
            return units[n] + " "
        elif n < 20:
            return teens[n - 10] + " "
        else:
            return tens[n // 10] + " " + units[n % 10] + " "

    def convert_hundreds(n):
        return hundreds[n // 100] + " " + get_hundreds_part(n % 100)

    result = []
    if number >= 1000:
        thousands_part = number // 1000
        result.append(convert_hundreds(thousands_part) + thousands[thousands_part % 10])
        number %= 1000

    if number > 0:
        result.append(convert_hundreds(number))

    return ' '.join(result).strip()


In [52]:
# Функция получения числа прописью
def number_to_words(number):
    return num2words(number, lang='ru')

In [53]:
# Функция преобразования года (в каком)
def year_to_ordinal_text(year):
    # Словари для чисел и их подрядковых форм
    base_numbers = {
        1: 'первом', 2: 'втором', 3: 'третьем', 4: 'четвёртом', 5: 'пятом',
        6: 'шестом', 7: 'седьмом', 8: 'восьмом', 9: 'девятом', 10: 'десятом',
        11: 'одиннадцатом', 12: 'двенадцатом', 13: 'тринадцатом', 14: 'четырнадцатом',
        15: 'пятнадцатом', 16: 'шестнадцатом', 17: 'семнадцатом', 18: 'восемнадцатом',
        19: 'девятнадцатом'
    }

    tens = {
        20: 'двадцатом', 30: 'тридцатом', 40: 'сороковом', 50: 'пятидесятом',
        60: 'шестидесятом', 70: 'семидесятом', 80: 'восьмидесятом', 90: 'девяностом'
    }

    hundreds = {
        100: 'сотом', 200: 'двухсотом', 300: 'трёхсотом', 400: 'четырёхсотом',
        500: 'пятисотом', 600: 'шестисотом', 700: 'семисотом', 800: 'восьмисотом',
        900: 'девятисотом'
    }

    if year < 1900 or year > 2099:
        return "Поддерживаются годы только от 1900 до 2099."

    if year < 2000:
        text = "тысяча "
        century_part = year // 100 % 10 * 100
        decade_part = year % 100
        century_text=hundreds.get(century_part, "")
    else:
        text = ""
        century_text="две тысячи "
        decade_part=year%100

    if decade_part == 0:
        pass
    elif decade_part in base_numbers:
        text += century_text+" "+base_numbers[decade_part]
    else:
        tens_part = decade_part // 10 * 10
        units_part = decade_part % 10
        text += century_text+" "
        text += tens.get(tens_part, "")+" "
        text += base_numbers.get(units_part, "")

    return text.strip()

#### Функция обработки текста

In [54]:
# Функции очистки текста
def clear_text(text: str,
               is_lower: bool = True,
               is_e_yo_replace: bool = True
               ) -> str:
    '''Функция очистки текста'''
    val: str = text
    if is_lower:
        val = val.lower()
    val = val \
        .replace("\n", " ") \
        .replace("\r", " ") \
        .strip()
    if is_e_yo_replace:
        val: str = re.sub(r'[ёЁ]', 'е', val)
    return val

#### Функции обработки дат

In [55]:
# Очистка и преобразование дат
def clear_date(text: str, is_delete: bool = False) -> str:
    '''Очистка и преобразование дат'''
    # pattern_1: re.Pattern = re.compile(CLEAR_DATA_PATTERN_1)
    # pattern_2: re.Pattern = re.compile(CLEAR_DATA_PATTERN_2)
    # pattern_3: re.Pattern = re.compile(CLEAR_DATA_PATTERN_3)
    pattern: re.Pattern = re.compile(CLEAR_DATA_PATTERN)
    if not pattern.findall(text):
        return text

    matches = list(pattern.finditer(text))
    for match in reversed(matches):
        match: re.Match = match
        val_str: str = ''

        # Если дату надо удалить, то удаляем
        if is_delete:
            text = replace_positions(text, match.start(), match.end(), val_str)
            continue

        # Тут надо как то обработать даты, но пока просто удаляем
        # if pattern_1.match(match.group()) and len(pattern_1.match(match.group()).group()) == len(match.group()):
        #     pass
        # elif pattern_2.match(match.group()) and len(pattern_2.match(match.group()).group()) == len(match.group()):
        #     pass
        # elif pattern_3.match(match.group()) and len(pattern_3.match(match.group()).group()) == len(match.group()):
        #     pass
        # else:
        #     pass

        text = replace_positions(text, match.start(), match.end(), val_str)
    return text

In [56]:
# Обработка указания только года
def clear_year(text: str, is_delete: bool = False) -> str:
    '''Очистка и преобразование года'''
    pattern: re.Pattern = re.compile(CLEAR_YEAR_PATTERN)
    if not pattern.findall(text):
        return text

    matches = list(pattern.finditer(text))
    for match in reversed(matches):
        match: re.Match = match
        val_str: str = ''
        # Если дату надо удалить, то удаляем
        if is_delete:
            text = replace_positions(text, match.start(), match.end(), val_str)
            continue

        # Если захватили год вместе с "в",
        # то надо будет вернуть эту "в"
        is_in_year_symbol: bool = False
        if 'в' in match.group().lower():
            is_in_year_symbol: bool = True

        year: int = int(re.sub(r'[^0-9]', '', match.group()))
        val_str: str = f'{" в" if is_in_year_symbol else ""}'
        val_str: str = f'{val_str} {year_to_ordinal_text(year)}'

        text = replace_positions(text, match.start(), match.end(), val_str)
    return text

#### Функции обработки сумм

In [57]:
# Функция преобразования сумм числами в прописные
def clear_amount(text: str, amount_currency: str) -> str:
    '''Функция преобразования сумм числами в прописные'''
    pattern: re.Pattern = re.compile(CLEAR_AMOUNT_PATTERN)
    if not pattern.findall(text):
        return text

    for match in reversed(list(pattern.finditer(text))):
        match: re.Match = match

        val_str: str = re.sub(r'[^0-9.,]+', '', match.group())
        val_str: str = val_str.strip('.,')

        if ',' in val_str:
            parts = re.split(r'[.,]', val_str)
            val_str = ''
            for i, p in enumerate(parts):
                val_str = f'{val_str}{p}'
                if i == len(parts) - 2:
                    val_str = f'{val_str}.'

        val: float = float(val_str)

        val_str: str = f"{number_to_words(val)} {amount_currency}"
        text = replace_positions(text, match.start(), match.end(), val_str)
    return text

#### Обработка паттерна оценки "n из m"

In [58]:
# Функция получения текста из паттерна: n из m
def clear_pattern_1(text: str, add_score: bool = False) -> str:
    '''Функция получения текста из паттерна: n из m'''

    def get_str_score(val: int, val_base: int) -> str:
        if not val_base:
            return ''
        perc: float = ((val / val_base) * 100)
        if val > val_base:
            return 'супер'
        if val == val_base or perc > 90:
            return 'отлично'
        if perc > 75:
            return 'хорошо'
        if perc > 50:
            return 'нормально'
        return 'плохо'

    pattern: re.Pattern = re.compile(CLEAR_TEXT_PATTERN_1)
    if not pattern.findall(text):
        return text

    for match in reversed(list(pattern.finditer(text))):
        match: re.Match = match
        vals: list[int] = [int(re.sub(r'[^0-9]+', '', v)) for v in str(match.group()).split("из")]
        val_str: str = f'{number_to_words(vals[0])} из {number_to_words_genitive(vals[1])}'

        if add_score:
            val_str: str = f'{val_str} {get_str_score(vals[0], vals[1])}'
        text = replace_positions(text, match.start(), match.end(), val_str)
    return text

#### Обработка паттерна "n в число"

In [59]:
# Функция получения текста из паттерна: n (число)
def clear_pattern_2(text: str) -> str:
    '''Функция получения текста из паттерна: n (число)'''

    pattern: re.Pattern = re.compile(CLEAR_TEXT_PATTERN_2)
    if not pattern.findall(text):
        return text

    for match in reversed(list(pattern.finditer(text))):
        match: re.Match = match

        val_str: str = f'{number_to_words(int(match.group()))}'

        text = replace_positions(text, match.start(), match.end(), val_str)
    return text

#### Финальная (грубая зачистка)

In [60]:
# Функция финальной (грубой) зачистки
def final_clear(text: str) -> str:
    '''Финальная (грубая) зачистка'''
    text = re.sub(CLEAR_TEXT_REPLACER, ' ', text)
    # text = re.sub(r"\s+", " ", text).strip()
    return text

#### Демо тест очистки данных

In [61]:
s = '''
    Хороший сервис, быстро отремонтировали подвеску на Мерседесе! 4 из 3 И все по честному, не навязывают лишних услуг! Спасибо)\n\n\n
    17.12.2022г. - Ого. Отзыв собрал 1000 просмотров. Но больше удивляет, что в 2018 году у меня оказывается был Мерседес (прав к тому году не было).
    И я что то чинил и мне понравиллсь. Во дела. Походу сервис так себе, раз лайки накручивают. Аккуратнее! \n\nНу и судя по комментариям организации,
    там не совсем вменяемые ребята).\n\n26.03.23г Не просто невменяемые, а ещё  и клоуны. Я тоже с вашей шарагой, благо не знаком.
    Тем более не мог оставлять Вам отзывы не являясь владельцем ТС. Судя по тому, что публикуете у себя левые отзывы, си реальными проблема.
    А так как вы позволяете себе общаться и пытаться какие диагнозы ставить - это о многом говорит. Стоило это 3.4 рублей 313,41руб
    '''

s: str = clear_text(text=s, is_lower=IS_LOWER_TEXT, is_e_yo_replace=IS_E_YO_REPLACE)

if IS_DATE_CLEAR_ENABLE:
    s: str = clear_date(text=s, is_delete=IS_DATE_ONLY_DELETE)

if IS_YEAR_CLEAR_ENABLE:
    s: str = clear_year(text=s, is_delete=IS_YEAR_ONLY_DELETE)

if IS_AMOUNT_CLEAR_ENABLE:
    s: str = clear_amount(text=s, amount_currency=AMOUNT_CLEAR_CURRENCY)

if IS_PATTERN_N_FROM_M_ENABLE:
    s: str = clear_pattern_1(text=s, add_score=IS_ADD_SCORE_TO_N_FROM_M)

if IS_PATTERN_N_TO_STR_ENABLE:
    s: str = clear_pattern_2(text=s)

if IS_FINAL_CLEAR_ENABLE:
    s: str = final_clear(text=s)

print(s)

Хороший сервис, быстро отремонтировали подвеску на Мерседесе  четыре из трех супер И все по честному, не навязывают лишних услуг  Спасибо            Ого. Отзыв собрал одна тысяча просмотров. Но больше удивляет, что  в две тысячи  восемнадцатом у меня оказывается был Мерседес  прав к тому году не было .     И я что то чинил и мне понравиллсь. Во дела. Походу сервис так себе, раз лайки накручивают. Аккуратнее    Ну и судя по комментариям организации,     там не совсем вменяемые ребята .   Не просто невменяемые, а еще  и клоуны. Я тоже с вашей шарагой, благо не знаком.     Тем более не мог оставлять Вам отзывы не являясь владельцем ТС. Судя по тому, что публикуете у себя левые отзывы, си реальными проблема.     А так как вы позволяете себе общаться и пытаться какие диагнозы ставить   это о многом говорит. Стоило это три целых четыре десятых руб. триста тринадцать целых сорок одна сотая руб.


#### Запуск очистки данных

In [67]:
# Очистка и подготовка данных
cleared_text: pd.Series = df_data_clear['text'] \
    .progress_apply(clear_text, args=(
        IS_LOWER_TEXT, IS_E_YO_REPLACE))

if IS_DATE_CLEAR_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(clear_date, args=(
            IS_DATE_ONLY_DELETE,))

if IS_YEAR_CLEAR_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(clear_year, args=(
            IS_YEAR_ONLY_DELETE,))

if IS_AMOUNT_CLEAR_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(clear_amount, args=(
            AMOUNT_CLEAR_CURRENCY,))

if IS_PATTERN_N_FROM_M_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(clear_pattern_1, args=(
            IS_ADD_SCORE_TO_N_FROM_M,))

if IS_PATTERN_N_TO_STR_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(clear_pattern_2)

if IS_FINAL_CLEAR_ENABLE:
    cleared_text: pd.Series = cleared_text \
        .progress_apply(final_clear)

# Возрат данных в датасет
df_data_clear['cleared_text'] = cleared_text
df_data_clear.head(5)

100%|██████████| 499800/499800 [00:01<00:00, 285839.42it/s]


,rating,rubrics,text,rating_int,rubrics_list,cleared_text
0,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...,3,[жилой комплекс],Московский квартал два. Шумно летом по ноча...
1,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...",5,"[магазин продуктов, продукты глубокой заморозк...","Замечательная сеть магазинов в общем, хороший ..."
2,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...",1,[фитнес клуб],"Не знаю смутят ли кого то данные правила, но я..."
3,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...,4,"[пункт проката, прокат велосипедов, сапсёрфинг]",Хорошие условия аренды. Дружелюбный персонал...
4,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,5,"[салон красоты, визажисты, стилисты, салон бро...",Топ мастер Ангелина топ во всех смыслах Немн...


In [63]:
# Инфо датасета
df_data_clear.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499800 entries, 0 to 499999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   rating        499800 non-null  object
 1   rubrics       499800 non-null  object
 2   text          499800 non-null  object
 3   rating_int    499800 non-null  int64 
 4   rubrics_list  499800 non-null  object
 5   cleared_text  499800 non-null  object
dtypes: int64(1), object(5)
memory usage: 26.7+ MB


## Сохранение датасета

In [64]:
# Сохранение обработанных данных
# Убрали столбцы, которые больше не нужны
df_data_clear \
    .drop(['rating'], axis=1) \
    .drop(['rubrics'], axis=1) \
    .drop(['text'], axis=1) \
    .to_csv(
        os.path.join(CLEARED_DATASET_PATH, 'cleared_dataset.csv'),
        sep=';',
        encoding='utf-8-sig',
        index_label='indx')